# 텐서플로 2 시작 - 초보용

Keras를 사용하여 다음을 수행합니다.

1. 사전에 빌드한 데이터세트를 로드합니다.
2. 이미지를 분류하는 신경망 머신 러닝 모델을 빌드합니다.
3. 이 신경망을 훈련합니다.
4. 모델의 정확도를 평가합니다.

## Tensorflow 설정

Tensorflow 프로그램 가져옴

GPU를 사용하려면 NVIDIA 제어판에서 3D 설정관리 > 전역 설정 > 고성능 NVIDIA 프로세서로 선택, 확인 후 진행필요

In [1]:
import tensorflow as tf

f'Tensorflow version : {tf.__version__}'

'Tensorflow version : 2.10.0'

In [2]:
from tensorflow.python.client import device_lib

# gpus = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13335348617137429812
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 2234830030
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10480590520549814578
 physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5"
 xla_global_id: 416903419]

## 데이터세트 로드

MNIST 데이테세트. 이미지의 픽셀 값 범위는 0~255. 이때 값을 255.0으로 나누어 0~1 범위로 조정

In [29]:
# MNIST 데이터세트
mnist = tf.keras.datasets.mnist

In [30]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0

## 머신 러닝 모델 빌드

tf.keras.Sequential 모델을 빌드

In [10]:
# Sequential은 각 레이어에 하나의 입력 텐서와 하나의 출력 텐서가 있는 레이어를 쌓는 데 유용
# 이 모델은 Flatten, Dense, Dropout 레이어를 사용

In [31]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Flatten(input_shape=(28, 28)), 			# 입력 레이어
	tf.keras.layers.Dense(128, activation='relu'),			# 은닉 레이어
	tf.keras.layers.Dropout(0.2),
	tf.keras.layers.Dense(10, activation='softmax')
])

In [32]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
predictions = model(X_train[:1]).numpy()
predictions

array([[0.06818771, 0.05678748, 0.12137134, 0.08634177, 0.1963781 ,
        0.17659378, 0.05585383, 0.04812076, 0.09890632, 0.09145894]],
      dtype=float32)

In [34]:
# tf.nn.softmax 함수는 이러한 로짓을 각 클래스에 대한 확률로 변환
tf.nn.softmax(predictions).numpy()

array([[0.09675398, 0.09565723, 0.102039  , 0.0985265 , 0.10998696,
        0.10783233, 0.09556796, 0.09483177, 0.09977225, 0.09903197]],
      dtype=float32)

In [37]:
# losses.SparseCategoricalCrossentropy를 사용하여 훈련용 손실 함수를 정의
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [38]:
loss_fn(y_train[:1], predictions).numpy()

2.2271776

In [39]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## 모델 훈련 및 평가

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0' ## 초기화할 GPU number

with tf.Graph().as_default():
    # GPU 메모리를 전부 할당하지 않고, 아주 적은 비율만 할당되어 시작됨. 프로세스의 메모리 수요에 따라 자동적으로 증가
	# 단, GPU 메모리를 처음부터 전체 비율을 사용하지 않음
	gpu_options = tf.compat.v1.GPUOptions(allow_growth=True) # V1 ???

In [40]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5


c:\Users\perso\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1875/1875 [==============================] - 12s 6ms/step - loss: 0.2995 - accuracy: 0.9128
Epoch 2/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.1446 - accuracy: 0.9568
Epoch 3/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.1070 - accuracy: 0.9677
Epoch 4/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0875 - accuracy: 0.9729
Epoch 5/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0747 - accuracy: 0.9762


In [22]:
print('GPU train') # 여기에선 GPU를 사용한다고 무지 빨라지지는 않는다.
with tf.device('/device:GPU:0'):
	model.fit(X_train, y_train, epochs=5)
 
print("CPU Train")
with tf.device('/device:CPU:0'):
	model.fit(X_train, y_train, epochs=5)

GPU train
Epoch 1/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0408 - accuracy: 0.9863
Epoch 2/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0386 - accuracy: 0.9868
Epoch 3/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0366 - accuracy: 0.9876
Epoch 4/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0333 - accuracy: 0.9884
Epoch 5/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0322 - accuracy: 0.9892
CPU Train
Epoch 1/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0297 - accuracy: 0.9895
Epoch 2/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0287 - accuracy: 0.9905
Epoch 3/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0281 - accuracy: 0.9899
Epoch 4/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0246 - accuracy: 0.9914
Epoch 5/5
1875/1875 [==============================] - 13s 7ms

In [27]:
model.evaluate(X_test,  y_test, verbose=2)

c:\Users\perso\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


313/313 - 2s - loss: 0.0730 - accuracy: 0.9778 - 2s/epoch - 6ms/step


[0.07300934195518494, 0.9778000116348267]

In [28]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [30]:
probability_model(X_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0.0853425 , 0.08534237, 0.08534317, 0.0853736 , 0.08534235,
        0.08534237, 0.08534235, 0.23188062, 0.0853425 , 0.08534816],
       [0.08533692, 0.08533744, 0.23196654, 0.08533736, 0.08533692,
        0.08533693, 0.08533692, 0.08533692, 0.08533712, 0.08533692],
       [0.08534072, 0.23190624, 0.08534271, 0.08534084, 0.08534228,
        0.08534089, 0.08534091, 0.08535917, 0.08534544, 0.08534074],
       [0.23192276, 0.08533968, 0.08534929, 0.08533983, 0.08533969,
        0.08534286, 0.08534641, 0.08533981, 0.08533972, 0.08533994],
       [0.08537843, 0.0853776 , 0.08537785, 0.0853776 , 0.23132145,
        0.08537763, 0.08537904, 0.08537807, 0.08537814, 0.08565414]],
      dtype=float32)>

### 결론

축하합니다! Keras API를 사용하는 사전에 빌드한 데이터세트를 사용하여 머신 러닝 모델을 훈련했습니다.